In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

from mimiciii_db import DB
from mimiciii_db.config import db_url

In [ ]:
db = DB.from_url(db_url())
print("Database connected successfully!")

In [ ]:
query = """
SELECT * FROM elixhauser_quan
"""

# Use your database connection
elixhauser_df = db.query_df(query)
elixhauser_df

In [ ]:
# USING THE LCA RESULTS CSV FILE FOR NOW. CHANGE TO DATABASE QUERY LATER.
lca_df = pd.read_csv('../temp/lca_all_subgroups.csv')
lca_df

In [ ]:
merged = pd.merge(
    elixhauser_df,
    lca_df[['hadm_id', 'subgroup_K6']],  # only need the 6-class assignment
    on='hadm_id',
    how='inner'
)
merged = merged.rename(columns={'subgroup_K6': 'subgroup_id'})
print(f"Merged shape: {merged.shape}")

In [ ]:
# All comorbidity columns (exclude hadm_id and subgroup)
ELIX_COLS = [c for c in merged.columns if c not in ['hadm_id', 'subgroup_id']]

# Overall prevalence (percent)
overall_prev = merged[ELIX_COLS].mean() * 100
overall_prev = overall_prev.sort_values(ascending=False)

# By subgroup
by_group = (
    merged.groupby('subgroup_id')[ELIX_COLS]
          .mean()
          .mul(100)
          .reindex(columns=overall_prev.index)  # consistent order
)
by_group.head()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# --- exact circular order---
paper_order = [
    'other_neurological','coagulopathy','cardiac_arrhythmias','hypertension',
    'fluid_electrolyte','congestive_heart_failure','diabetes_uncomplicated',
    'chronic_pulmonary','pulmonary_circulation','peripheral_vascular',
    'hypothyroidism','valvular_disease','renal_failure','obesity',
    'diabetes_complicated','metastatic_cancer','aids','psychoses',
    'blood_loss_anemia','peptic_ulcer','lymphoma','solid_tumor',
    'rheumatoid_arthritis','weight_loss','paralysis','deficiency_anemias',
    'drug_abuse','alcohol_abuse','liver_disease','depression'
]

# pretty labels
def nice(s):
    mapping = {
        'fluid_electrolyte':'fluid electrolyte disorder',
        'chronic_pulmonary':'chronic pulmonary disease',
        'pulmonary_circulation':'pulmonary circulation disorder',
        'peripheral_vascular':'peripheral vascular disease',
        'other_neurological':'other neurological disorder',
        'diabetes_uncomplicated':'diabetes uncomplicated',
        'diabetes_complicated':'diabetes complicated',
        'blood_loss_anemia':'blood loss anemia',
        'deficiency_anemias':'deficiency anemias',
        'congestive_heart_failure':'congestive heart failure',
        'cardiac_arrhythmias':'cardiac arrhythmias',
    }
    return mapping.get(s, s.replace('_',' '))
labels = [nice(c) for c in paper_order]

# -------- build subgroup prevalences (same order) --------
merged = (
    elixhauser_df
    .merge(lca_df[['hadm_id','subgroup_K6']], on='hadm_id', how='inner')
    .rename(columns={'subgroup_K6':'subgroup'})
)
ELIX = [c for c in merged.columns if c not in ['hadm_id','subgroup']]
by_sub = (merged.groupby('subgroup')[ELIX].mean()*100).reindex(columns=paper_order)

# optional colors per subgroup (roughly like the paper)
palette = {1:'#4d4d4d', 2:'#ff7b7b', 3:'#4fd37a', 4:'#4f6bed', 5:'#9fe2ff', 6:'#ff66cc'}

def plot_subgroup(vals_series, title, color='#4f4f4f'):
    # --- your Fig 3A style (tangential labels + connectors) ---
    vals  = np.clip(vals_series.reindex(paper_order).values, 0, 50)  # cap at 50%
    n     = len(vals)
    theta = np.linspace(0, 2*np.pi, n, endpoint=False)
    width = (2*np.pi)/n * 0.92
    r_in  = 14

    plt.figure(figsize=(8,8))
    ax = plt.subplot(111, polar=True)
    ax.grid(False)
    ax.spines['polar'].set_visible(False)
    ax.set_ylim(0, r_in+50)
    ax.set_theta_offset(np.pi/2)
    ax.set_theta_direction(-1)
    ax.set_yticklabels([])

    # bars
    ax.bar(theta, vals, width=width, bottom=r_in,
           color=color, edgecolor='white', linewidth=1.0, alpha=0.95)

    # inner ring
    ring = plt.Circle((0,0), r_in, transform=ax.transData._b,
                      fill=False, lw=1.5, color='black')
    ax.add_artist(ring)

    # tangential labels
    ax.set_xticks(theta + width/2)
    ax.set_xticklabels(labels, fontsize=8)
    ax.tick_params(pad=18)
    for lab, ang in zip(ax.get_xticklabels(), theta + width/2):
        deg = np.degrees(ang)
        lab.set_rotation(deg - 90)
        lab.set_rotation_mode('anchor')
        lab.set_horizontalalignment('right' if 90 <= deg <= 270 else 'left')
        lab.set_verticalalignment('center')

    # connector lines from bar tips to label 
    r_label_ring = r_in + 50 + 1.0
    for ang, val in zip(theta + width/2, vals):
        r_tip = r_in + val
        ax.plot([ang, ang], [r_tip, r_label_ring],
                color='0.6', lw=0.8, solid_capstyle='round', zorder=0)

    ax.set_title(title, y=1.08, fontsize=12)
    plt.tight_layout()
    plt.show()

# -------- plot Subgroups 1-6 --------
for sg in range(1, 7):
    plot_subgroup(by_sub.loc[sg], title=f"Subgroup {sg}",
                  color=palette.get(sg, '#4f4f4f'))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ---------- exact circular order (paper) ----------
paper_order = [
    'other_neurological','coagulopathy','cardiac_arrhythmias','hypertension',
    'fluid_electrolyte','congestive_heart_failure','diabetes_uncomplicated',
    'chronic_pulmonary','pulmonary_circulation','peripheral_vascular',
    'hypothyroidism','valvular_disease','renal_failure','obesity',
    'diabetes_complicated','metastatic_cancer','aids','psychoses',
    'blood_loss_anemia','peptic_ulcer','lymphoma','solid_tumor',
    'rheumatoid_arthritis','weight_loss','paralysis','deficiency_anemias',
    'drug_abuse','alcohol_abuse','liver_disease','depression'
]

def nice(s):
    mapping = {
        'fluid_electrolyte':'fluid electrolyte disorder',
        'chronic_pulmonary':'chronic pulmonary disease',
        'pulmonary_circulation':'pulmonary circulation disorder',
        'peripheral_vascular':'peripheral vascular disease',
        'other_neurological':'other neurological disorder',
        'diabetes_uncomplicated':'diabetes uncomplicated',
        'diabetes_complicated':'diabetes complicated',
        'blood_loss_anemia':'blood loss anemia',
        'deficiency_anemias':'deficiency anemias',
        'congestive_heart_failure':'congestive heart failure',
        'cardiac_arrhythmias':'cardiac arrhythmias',
    }
    return mapping.get(s, s.replace('_',' '))
labels = [nice(c) for c in paper_order]

# ---------- subgroup prevalences (RAW %) ----------
merged = (
    elixhauser_df
    .merge(lca_df[['hadm_id','subgroup_K6']], on='hadm_id', how='inner')
    .rename(columns={'subgroup_K6':'subgroup'})
)
ELIX = [c for c in merged.columns if c not in ['hadm_id','subgroup']]
by_sub = (merged.groupby('subgroup')[ELIX].mean()*100).reindex(columns=paper_order)

# optional colors per subgroup (pick your own if you like)
palette = {1:'#4d4d4d', 2:'#ff7b7b', 3:'#4fd37a', 4:'#4f6bed', 5:'#9fe2ff', 6:'#ff66cc'}

# ---------- helper to draw one wheel on a given axes ----------
def draw_wheel(ax, series_vals, title, color='#4f4f4f'):
    vals  = np.clip(series_vals.reindex(paper_order).values, 0, 50)  # cap 50%
    n     = len(vals)
    theta = np.linspace(0, 2*np.pi, n, endpoint=False)
    width = (2*np.pi)/n * 0.92
    r_in  = 10  # inner radius a bit smaller to fit in subplots

    # clean polar
    ax.grid(False)
    ax.spines['polar'].set_visible(False)
    ax.set_theta_offset(np.pi/2)
    ax.set_theta_direction(-1)
    ax.set_yticklabels([])
    ax.set_ylim(0, r_in+50)

    # bars
    ax.bar(theta, vals, width=width, bottom=r_in,
           color=color, edgecolor='white', linewidth=1.0, alpha=0.95)

    # inner ring
    ring = plt.Circle((0,0), r_in, transform=ax.transData._b,
                      fill=False, lw=1.2, color='black')
    ax.add_artist(ring)

    # tangential labels (same style as your Fig 3A)
    ax.set_xticks(theta + width/2)
    ax.set_xticklabels(labels, fontsize=7)
    ax.tick_params(pad=10)
    for lab, ang in zip(ax.get_xticklabels(), theta + width/2):
        deg = np.degrees(ang)
        lab.set_rotation(deg - 90)
        lab.set_rotation_mode('anchor')
        lab.set_horizontalalignment('right' if 90 <= deg <= 270 else 'left')
        lab.set_verticalalignment('center')

    # short connector lines from bar tips to label ring
    r_label_ring = r_in + 50 + 0.8
    for ang, v in zip(theta + width/2, vals):
        r_tip = r_in + v
        ax.plot([ang, ang], [r_tip, r_label_ring],
                color='0.6', lw=0.7, solid_capstyle='round', zorder=0)

    ax.set_title(title, y=1.05, fontsize=12)

# ---------- create 2×3 grid like the paper ----------
fig, axes = plt.subplots(2, 3, subplot_kw={'projection':'polar'}, figsize=(14,9))

sg = 1
for i in range(2):
    for j in range(3):
        draw_wheel(axes[i, j], by_sub.loc[sg], title=f"Subgroup {sg}",
                   color=palette.get(sg, '#4f4f4f'))
        sg += 1

plt.tight_layout()
plt.show()
